In [1]:
import pandas as pd; import numpy as np; import janitor as jn; from pandas import Timedelta

In [2]:
df = pd.DataFrame(dict(B=[2.0,0.0]))
right=pd.DataFrame(dict(Numeric=[1.0]))

In [3]:
df

,B
0,2.0
1,0.0


In [4]:
right

,Numeric
0,1.0


In [5]:
df[["B"]].conditional_join(
        right[["Numeric"]],
        ("B", "Numeric", "!="),
        how="inner",
        keep="last",

        sort_by_appearance=False,
    )

,B,Numeric
0,0.0,1.0
1,2.0,1.0


In [6]:
from string import ascii_lowercase
np.random.seed(1)
n = 20; k = 30
mapp = {k:v for v,k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high = 100, size = n)
idx2 = np.random.randint(0, high = 100, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2),
          )


d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [7]:
d1

,x,start,end,xx
0,a,29,37,0
1,a,12,14,0
2,d,50,72,3
3,c,9,68,2
4,b,75,87,1
5,a,5,87,0
6,d,79,94,3
7,b,64,96,1
8,b,16,86,1
9,d,1,13,3


In [8]:
d2

,x,pos1,xx
0,c,131,2
1,a,134,0
2,e,122,4
3,j,106,9
4,c,92,2
5,h,148,7
6,h,83,7
7,k,115,10
8,j,125,9
9,i,137,8


In [9]:
d2.sort_values(['pos1','xx'])

,x,pos1,xx
11,j,60,9
17,e,62,4
10,g,63,6
13,h,66,7
21,d,67,3
27,h,80,7
19,f,81,5
6,h,83,7
28,h,90,7
4,c,92,2


In [10]:
bla = (d1
.conditional_join(
    d2, 
    ('start', 'pos1', '>='), 
    ('end', 'pos1', '>='), 
    ('xx', 'xx', '<'), 
    use_numba=True)
)

bla

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


left              right        
      x start end xx     x pos1 xx
0     b    75  87  1     d   67  3
1     b    75  87  1     h   66  7
2     b    75  87  1     g   63  6
3     b    75  87  1     e   62  4
4     b    75  87  1     j   60  9
5     d    79  94  3     h   66  7
6     d    79  94  3     g   63  6
7     d    79  94  3     e   62  4
8     d    79  94  3     j   60  9
9     b    64  96  1     g   63  6
10    b    64  96  1     e   62  4
11    b    64  96  1     j   60  9

In [11]:
(d1
.conditional_join(
    d2, 
    ('start', 'pos1', '>='), 
    ('end', 'pos1', '>='), 
    ('xx', 'xx', '<'), 
    use_numba=False)
)

left              right        
      x start end xx     x pos1 xx
0     b    75  87  1     j   60  9
1     b    75  87  1     e   62  4
2     b    75  87  1     g   63  6
3     b    75  87  1     h   66  7
4     b    75  87  1     d   67  3
5     d    79  94  3     j   60  9
6     d    79  94  3     e   62  4
7     d    79  94  3     g   63  6
8     d    79  94  3     h   66  7
9     b    64  96  1     j   60  9
10    b    64  96  1     e   62  4
11    b    64  96  1     g   63  6

In [12]:
# pprint([region_left, region_right, left_region, right_region])

In [13]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})
                    
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [14]:
# df1 = pd.concat([df1]*10)
# df2 = pd.concat([df2]*10)

In [15]:
(df1
.conditional_join(
    df2,
    ('id', 'id', '>'),
    ('value_1', 'value_2A', '<='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))

left         right                  
    id value_1    id value_2A value_2B
0    2       1     1       12       15
1    2       1     1        7        9
2    2       1     1        3        5
3    2       3     1       12       15
4    2       3     1        7        9
5    2       3     1        3        5
6    3       4     1       12       15
7    3       4     1        7        9

In [16]:
df1 = pd.concat([df1]*2)
df2 = pd.concat([df2]*2)

In [17]:
df2.sort_values(['value_2A','value_2B'],kind='stable')

,id,value_2A,value_2B
0,1,0,1
4,2,0,1
0,1,0,1
4,2,0,1
7,3,1,3
7,3,1,3
5,2,2,4
5,2,2,4
1,1,3,5
1,1,3,5


In [18]:
bla = (df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
    # ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))

bla

left         right                  
      id value_1    id value_2A value_2B
0      1       2     3        1        3
1      1       2     3        1        3
2      1       2     2        2        4
3      1       2     2        2        4
4      1       2     1        3        5
..   ...     ...   ...      ...      ...
119    3       4     2        3        6
120    3       4     1        7        9
121    3       4     1        7        9
122    3       4     1       12       15
123    3       4     1       12       15

[124 rows x 5 columns]

In [19]:
%%timeit
(df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
    # ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))


1.13 ms ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%%timeit
(df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
    # ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=False
))

924 µs ± 2.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
(df1
.conditional_join(
    df2,
    # ('id', 'id', '>'),
    # ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=False
))

left         right                  
      id value_1    id value_2A value_2B
0      1       2     3        1        3
1      1       2     3        1        3
2      1       2     2        2        4
3      1       2     2        2        4
4      1       2     1        3        5
..   ...     ...   ...      ...      ...
119    3       4     2        3        6
120    3       4     1        7        9
121    3       4     1        7        9
122    3       4     1       12       15
123    3       4     1       12       15

[124 rows x 5 columns]

In [22]:
url='/Users/samuel.oranyeli/events.csv'
#url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
events = pd.read_csv(url, parse_dates=['start', 'end'])
events.head()

,id,name,audience,start,sponsor,end
0,1,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,2,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,3,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,4,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,5,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [23]:
bla =(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '<'),
    ('audience','audience','>'),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

bla

left                                          right                      \
        id               start                 end     id               start   
0       15 1993-04-04 16:00:00 1993-04-04 18:00:00  11178 1993-04-04 17:00:00   
1       17 2030-10-25 07:00:00 2030-10-25 07:27:00  19605 2030-10-25 06:00:00   
2       26 2005-10-04 17:00:00 2005-10-04 17:18:00   8218 2005-10-04 17:00:00   
3       35 2024-05-02 15:00:00 2024-05-02 15:35:00   6916 2024-05-02 15:00:00   
4       40 2016-03-05 00:00:00 2016-03-05 00:08:00   7993 2016-03-05 00:00:00   
..     ...                 ...                 ...    ...                 ...   
934  28578 2029-02-05 04:00:00 2029-02-05 04:47:00  29313 2029-02-05 04:00:00   
935  28667 2014-05-13 06:00:00 2014-05-13 08:00:00  29784 2014-05-13 06:00:00   
936  28924 2023-09-11 07:00:00 2023-09-11 07:10:00  29672 2023-09-11 05:00:00   
937  29105 2022-11-21 20:00:00 2022-11-21 22:00:00  29762 2022-11-21 21:00:00   
938  29375 2000-08-26 13:00:00 2000-08-26 13:53:00  29966 2000-08-26 11:00:00   

                         
                    end  
0   1993-04-04 17:22:00  
1   2030-10-25 08:00:00  
2   2005-10-04 17:27:00  
3   2024-05-02 15:36:00  
4   2016-03-05 00:42:00  
..                  ...  
934 2029-02-05 04:24:00  
935 2014-05-13 08:00:00  
936 2023-09-11 07:00:00  
937 2022-11-21 23:00:00  
938 2000-08-26 13:00:00  

[939 rows x 6 columns]

In [24]:
events.shape[0]**2

899940001

In [25]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    # ('id', 'id', '<'),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)


17.8 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    # ('id', 'id', '<'),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)


67.5 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '<'),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

left                                          right  \
         id               start                 end     id   
0        10 1993-11-27 12:00:00 1993-11-27 12:37:00   2345   
1        15 1993-04-04 16:00:00 1993-04-04 18:00:00  11178   
2        17 2030-10-25 07:00:00 2030-10-25 07:27:00  19605   
3        26 2005-10-04 17:00:00 2005-10-04 17:18:00   8218   
4        35 2024-05-02 15:00:00 2024-05-02 15:35:00   6916   
...     ...                 ...                 ...    ...   
1846  28578 2029-02-05 04:00:00 2029-02-05 04:47:00  29313   
1847  28667 2014-05-13 06:00:00 2014-05-13 08:00:00  29784   
1848  28924 2023-09-11 07:00:00 2023-09-11 07:10:00  29672   
1849  29105 2022-11-21 20:00:00 2022-11-21 22:00:00  29762   
1850  29375 2000-08-26 13:00:00 2000-08-26 13:53:00  29966   

                                              
                   start                 end  
0    1993-11-27 10:00:00 1993-11-27 12:00:00  
1    1993-04-04 17:00:00 1993-04-04 17:22:00  
2    2030-10-25 06:00:00 2030-10-25 08:00:00  
3    2005-10-04 17:00:00 2005-10-04 17:27:00  
4    2024-05-02 15:00:00 2024-05-02 15:36:00  
...                  ...                 ...  
1846 2029-02-05 04:00:00 2029-02-05 04:24:00  
1847 2014-05-13 06:00:00 2014-05-13 08:00:00  
1848 2023-09-11 05:00:00 2023-09-11 07:00:00  
1849 2022-11-21 21:00:00 2022-11-21 23:00:00  
1850 2000-08-26 11:00:00 2000-08-26 13:00:00  

[1851 rows x 6 columns]

In [28]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2]
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15]
       )
D=pd.DataFrame(D)

In [29]:
C = pd.concat([C]*20_000)
D = pd.concat([D]*1_000)

In [30]:
f=(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
 # ('unitsSold','unitsSold','<'),
    use_numba=True)
)
f

left                      right                     
           key vol profit unitsSold   key vol profit unitsSold
0           c1  35     45        15    d5  40     35        40
1           c1  35     45        15    d7  40     30         5
2           c1  35     45        15    d5  40     35        40
3           c1  35     45        15    d7  40     30         5
4           c1  35     45        15    d5  40     35        40
...        ...  ..    ...       ...   ...  ..    ...       ...
339999995   c7  17     11         2    d2  50     10        35
339999996   c7  17     11         2    d2  50     10        35
339999997   c7  17     11         2    d2  50     10        35
339999998   c7  17     11         2    d2  50     10        35
339999999   c7  17     11         2    d2  50     10        35

[340000000 rows x 8 columns]

In [31]:
f=(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
  ('unitsSold','unitsSold','<'),
  ('vol','profit','>'),
    use_numba=True)
)
f

left                      right                     
          key vol profit unitsSold   key vol profit unitsSold
0          c1  35     45        15    d2  50     10        35
1          c1  35     45        15    d2  50     10        35
2          c1  35     45        15    d2  50     10        35
3          c1  35     45        15    d2  50     10        35
4          c1  35     45        15    d2  50     10        35
...       ...  ..    ...       ...   ...  ..    ...       ...
99999995   c7  17     11         2    d2  50     10        35
99999996   c7  17     11         2    d2  50     10        35
99999997   c7  17     11         2    d2  50     10        35
99999998   c7  17     11         2    d2  50     10        35
99999999   c7  17     11         2    d2  50     10        35

[100000000 rows x 8 columns]

In [32]:
C.shape[0] * D.shape[0]

1120000000

In [33]:
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    ('unitsSold','unitsSold','<'),
    use_numba=False)
)

left                      right                     
           key vol profit unitsSold   key vol profit unitsSold
0           c1  35     45        15    d2  50     10        35
1           c1  35     45        15    d2  50     10        35
2           c1  35     45        15    d2  50     10        35
3           c1  35     45        15    d2  50     10        35
4           c1  35     45        15    d2  50     10        35
...        ...  ..    ...       ...   ...  ..    ...       ...
199999995   c7  17     11         2    d2  50     10        35
199999996   c7  17     11         2    d2  50     10        35
199999997   c7  17     11         2    d2  50     10        35
199999998   c7  17     11         2    d2  50     10        35
199999999   c7  17     11         2    d2  50     10        35

[200000000 rows x 8 columns]

In [34]:
%%timeit
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    ('unitsSold','unitsSold','<'),
    use_numba=True)
)

KeyboardInterrupt: 

In [ ]:
%%timeit
(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    ('unitsSold','unitsSold','<'),
    use_numba=False)
)

14.1 s ± 47.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
D

,key,vol,profit,unitsSold
0,d1,20,30,20
1,d2,50,10,35
2,d3,15,12,10
3,d4,16,52,12
4,d5,40,35,40
...,...,...,...,...
3,d4,16,52,12
4,d5,40,35,40
5,d6,20,20,30
6,d7,40,30,5


In [ ]:
C.sort_values('vol')

,key,vol,profit,unitsSold
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
...,...,...,...,...
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80


In [ ]:
D.sort_values('vol')
#D.sort_values('profit')
#D.sort_values('profit',ascending=False)


,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35


In [ ]:
D

,key,vol,profit,unitsSold
0,d1,20,30,20
1,d2,50,10,35
2,d3,15,12,10
3,d4,16,52,12
4,d5,40,35,40
...,...,...,...,...
3,d4,16,52,12
4,d5,40,35,40
5,d6,20,20,30
6,d7,40,30,5


In [ ]:
C.sort_values('vol')

,key,vol,profit,unitsSold
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
2,c3,5,55,30
...,...,...,...,...
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80
5,c6,90,55,80


In [ ]:
D.sort_values('vol')

,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35


In [ ]:
pd.concat([C.loc[:, ['key','vol']], D.loc[:, ['key','vol']]]).sort_values('vol',kind='stable')

,key,vol
7,d8,2
7,d8,2
7,d8,2
7,d8,2
7,d8,2
...,...,...
5,c6,90
5,c6,90
5,c6,90
5,c6,90


In [ ]:
C.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
2,c3,5,55,30
5,c6,90,55,80
2,c3,5,55,30
5,c6,90,55,80
5,c6,90,55,80
...,...,...,...,...
6,c7,17,11,2
6,c7,17,11,2
6,c7,17,11,2
6,c7,17,11,2


In [ ]:
D.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35


In [ ]:
C.vol.sort_values()

2     5
2     5
2     5
2     5
2     5
     ..
5    90
5    90
5    90
5    90
5    90
Name: vol, Length: 140000, dtype: int64

In [ ]:
C.vol.sort_values().searchsorted(D.vol, side='left')

array([ 80000, 120000,  20000, ...,  80000, 120000,      0])

In [ ]:
D.sort_values(['vol','profit', 'unitsSold'])

,key,vol,profit,unitsSold
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
7,d8,2,57,15
...,...,...,...,...
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
1,d2,50,10,35
